In [2]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import folium
import openrouteservice as ors
from geopy.distance import great_circle

In [3]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [4]:
def get_stations():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [5]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
stations_real_time


,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
0,"Avenida del Ensanche de Vallecas, 9,",10,17,2190,2,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440,"(-3.61253028, 40.37043968)"
1,"Paseo de la Chopera,33,Comunidad de Madrid España",15,8,2205,2,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
2,"Paseo de la Castellana nº 122,",19,4,2224,1,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100,"(-3.6908, 40.4491)"
3,"Paseo de la Castellana nº 164,Comunidad de Mad...",5,18,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137,"(-3.6894151, 40.4591366)"
4,"Guetaria , 84b,",18,6,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168,"(-3.71569102, 40.36916772)"
...,...,...,...,...,...,...,...,...,...,...,...,...
608,"Calle Francisco Balseiro nº 1,",0,0,1643,3,237 - Pablo Iglesias,237,0,06,-3.710525,40.451333,"(-3.710525, 40.451333)"
609,"237- calle Francisco Balseiro,Comunidad de Mad...",0,0,2362,3,1664 - 237- calle Francisco Balseiro,1664,0,06,-3.710274,40.451682,"(-3.7102736, 40.4516819)"
610,"Calle José Abascal frente al nº 2,",12,11,1607,2,201 - Canal,201,23,07,-3.703833,40.438694,"(-3.7038333, 40.4386944)"
611,"Calle Albelda, 6,",0,0,2316,3,"528 - Albelda, 6",528,24,15,-3.657367,40.449422,"(-3.6573666, 40.4494225)"


In [17]:
def get_district(df, district_number):
    result = df[df["code_district"] == district_number]
    return result

district_03 = get_district(stations_real_time, "03")
district_03

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
71,"Calle O'Donnell nº 28,",2,25,1465,0,61 - Narváez,61,27,03,-3.675314,40.421455,"(-3.6753139, 40.4214551)"
72,"Calle Ibiza nº 62,",11,12,1467,2,63 - Ibiza,63,23,03,-3.670896,40.417924,"(-3.6708959, 40.4179237)"
73,"Calle Antonio Maura nº 15,",20,4,1468,1,65 - Antonio Maura,65,24,03,-3.689425,40.416701,"(-3.689424797942769, 40.4167014469994)"
75,"Calle Espalter nº 1,",15,4,1470,1,68 - Espalter,68,19,03,-3.691323,40.412769,"(-3.6913232, 40.4127692)"
76,"Avenida de Alfonso XII nº 54,",24,3,1471,1,69 - Puerta del Ángel Caído,69,27,03,-3.688822,40.409808,"(-3.688822, 40.409808)"
77,"Avenida de Menéndez Pelayo nº 63,",10,17,1472,2,70 - Puerta del Doce de Octubre,70,27,03,-3.677873,40.415382,"(-3.677872542327878, 40.415382385574716)"
78,"Calle Doce de Octubre nº 28,",3,20,1473,0,71 - Doce de Octubre,71,23,03,-3.673900,40.416100,"(-3.6739, 40.4161)"
79,"Calle Doctor Esquerdo nº 99,",3,20,1474,0,72 - Sainz de Baranda,72,23,03,-3.669600,40.415900,"(-3.6696, 40.4159)"
80,"Avenida de Nazaret nº 7,",3,20,1475,0,73 - Plaza de los Astros,73,23,03,-3.668700,40.411500,"(-3.6687, 40.4115)"
81,"Avenida de Menéndez Pelayo nº 69,",11,16,1476,2,74 - Puerta del Pacífico,74,27,03,-3.676589,40.412063,"(-3.676588763788984, 40.41206290978386)"


In [6]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [7]:
district_02

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",15,8,2205,2,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
35,"Paseo Virgen del Puerto, 25,",0,0,2181,3,"BiciMAD5 - Paseo Virgen del Puerto, 25",BiciMAD5,0,02,-3.721009,40.411611,"(-3.72100859, 40.41161087)"
65,"Calle Ribera de Curtidores nº 28,",5,22,1450,0,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
113,Calle Teresa López Valcárcel esq. Teniente Cor...,12,12,2071,2,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
156,"Calle Delicias, 45,",11,8,2117,2,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819,"(-3.69091831, 40.40381914)"
209,"Glorieta de Embajadores nº 6,",12,12,1451,2,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
233,"PASEO DE LOS MELANCÓLICOS 73,",14,9,2247,2,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
247,"271 - Calle Amaltea , 1,",16,8,2265,2,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
249,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",5,17,2267,0,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,22,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
251,"Calle Méndez Alvaro, 73,",18,4,2269,1,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"


In [8]:
def get_light1(df):
    df_light1 = df[df["light"]==1]
    return df_light1

In [9]:
get_light1(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
251,"Calle Méndez Alvaro, 73,",18,4,2269,1,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
372,"Calle del tejo,",18,4,2305,1,272 - Calle del tejo,272,23,02,-3.679186,40.396630,"(-3.679186, 40.39663)"
407,"Calle Juan Martín el Empecinado nº 16,",17,6,1524,1,118 - Juan Martín,118,23,02,-3.688241,40.400781,"(-3.6882407, 40.400781)"
422,"Paseo Imperial nº 20,",24,3,1642,1,236 - Paseo Imperial,236,27,02,-3.717078,40.407579,"(-3.7170785, 40.4075794)"
530,"Calle Segovia nº 45,",20,7,1573,1,167 - Segovia 45,167,27,02,-3.717489,40.413748,"(-3.71748917032373, 40.41374770742206)"
532,"Calle Toledo nº 181,",22,2,1575,1,169 - Pirámides,169,24,02,-3.713823,40.401589,"(-3.713823, 40.401589)"
535,"Calle Bustamante nº 1,",19,3,1578,1,172 - Delicias,172,23,02,-3.693493,40.400828,"(-3.6934926, 40.4008279)"
540,"Calle Bolívar nº 3,",21,2,1583,1,177 - Legazpi,177,23,02,-3.694188,40.391447,"(-3.6941876374343896, 40.391447085678806)"
543,"Calle Retama nº 5,",23,0,1586,1,180 - Méndez Álvaro,180,23,02,-3.675890,40.394042,"(-3.67589, 40.3940423)"


In [10]:
def get_light0(df):
    df_light0 = df[df["light"]==0]
    return df_light0

In [11]:
get_light0(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
65,"Calle Ribera de Curtidores nº 28,",5,22,1450,0,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
249,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",5,17,2267,0,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,22,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
330,"Paseo virgen de puerto,",2,13,2304,0,277 - Paseo virgen de puerto,277,15,02,-3.721178,40.411622,"(-3.721178, 40.411622)"
409,"Calle Palos de la Frontera nº 27,",7,20,1526,0,120 - Palos de la Frontera,120,27,02,-3.694825,40.403090,"(-3.6948251018524214, 40.403090451430614)"
410,"Paseo de Santa María de la Cabeza nº 58,",7,20,1527,0,121 - Santa María de la Cabeza,121,27,02,-3.698603,40.401749,"(-3.6986027, 40.4017487)"
526,"Paseo de los Olmos nº 28,",3,20,1568,0,162 - Metro Pirámides,162,23,02,-3.710577,40.403497,"(-3.7105765, 40.4034968)"
536,"Paseo de la Esperanza nº 59,",6,18,1579,0,173 - Puente Praga,173,24,02,-3.702484,40.398347,"(-3.7024839, 40.3983467)"


In [12]:
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [16]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02).copy()
distrito02_high = get_light1(district_02).copy()

distrito02_high['visited'] = False
distrito02_low['visited'] = False

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

stop_counter = 1

for i in range(100):
    if van == "empty":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)].empty:
            nearest_station = find_nearest_to_coords(distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'visited'] = True
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with high occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='orange')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)].empty:
            nearest_station = find_nearest_to_coords(distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'visited'] = True
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with low occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='green')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
final_route = create_route(client, coords_list[-1], vehicle_start)
folium.PolyLine(locations=[coord[::-1] for coord in final_route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m